In [1]:
### 目的：解析 GPT的结果
import pickle 
import json 
import pandas as pd
import numpy as np
from tqdm import tqdm


/var/folders/56/b87bdh2x1kbcxmgsfhmgh1sh0000gn/T/ipykernel_4905/2617999452.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
### 读数据 - label数据
import json
with open('gpt4_res_parse_train_level.json', 'rb') as f:
    total_prompt_train = json.load(f)

with open('gpt4_res_parse_valid_level.json', 'rb') as f:
    total_prompt_valid = json.load(f)

with open('gpt4_res_parse_test_level.json', 'rb') as f:
    total_prompt_test = json.load(f)


In [3]:
### 目的：从GPT结果中解析出可以使用的数据
import json
import re

import json 
import pandas as pd
import numpy as np 
import re

def find_b_references(text):
    # 定义正则表达式模式，匹配'b'后跟着一个或多个数字
    text = text.lower()
    pattern = r'\bb\d+\b'
    
    # 使用findall方法查找所有匹配的子串
    # \b 是一个单词边界，防止匹配到单词中的一部分
    matches = re.findall(pattern, text)
    
    return matches

def parse_gemini_res(text):
    res = {}
    geimini_result_list = text.split('paperkey')
    for x in geimini_result_list:
        try:
            k, v = x.split('papervalue')
            res[k] = v
        except:
            pass
    return res

def replace_references(text):
    # 使用正则表达式查找所有的‘reference 数字’模式，并替换成‘[b数字]’格式
    replaced_text = re.sub(r'reference (\d+)', r'[b\1]', text)
    return replaced_text


def process_input(text):
    text = text.lower()
    text = text.replace('**direct inspiration', '. direct inspiration').replace('** direct inspiration', '. direct inspiration').replace('## direct inspiration', '. direct inspiration')
    text = text.replace('**indirect inspiration', '. indirect inspiration').replace('** indirect inspiration', '. indirect inspiration').replace('## indirect inspiration', '. indirect inspiration')
    text = text.replace('**other important', '. other important').replace('** other important', '. other important').replace('## other important', '. other important')
    text = replace_references(text)
    return text



def extract_references(input_text):
    inspiration_types = {'Summary' : None,
        's1': [],
        's2': [],
        's3': []
    }
    
    #提取第一部分
    #input_text = input_text.lower()
    input_text = process_input(input_text)
    text_step1 = input_text.split('"direct inspiration"')
    summary = text_step1[0]
    text_step2 = text_step1[1].split('"indirect inspiration"')
    text_step3 = text_step2[1].split('"other inspiration"')
    s1, s2, s3 = text_step2[0], text_step3[0], text_step3[1]
    inspiration_types['summary'] = summary
    inspiration_types['s1'] = find_b_references(s1)
    inspiration_types['s2'] = find_b_references(s2)
    inspiration_types['s3'] = find_b_references(s3)
    return inspiration_types

def extract_references_2(input_text):
    inspiration_types = {'Summary' : None,
        's1': [],
        's2': [],
        's3': []
    }
    input_text = input_text.lower()
    input_text = input_text.replace("directinspiration","direct inspiration").replace("indirectinspiration","indirect inspiration").replace("otherinspiration","other inspiration")
    try:
        parse_res = json.loads(input_text.strip('```json\n').strip('\n```'))
    except:
        try:
            parse_res = eval(input_text.strip('```json\n').strip('\n```'))
        except:
            return extract_references(input_text)
        
    try:
        if 'references' not in parse_res['direct inspiration'].keys():
            s1 = list(parse_res['direct inspiration'].keys())
            s2 = list(parse_res['indirect inspiration'].keys())
            s3 = list(parse_res['other inspiration'].keys())
            inspiration_types['s1'].extend(s1)
            inspiration_types['s2'].extend(s2)
            inspiration_types['s3'].extend(s3)
        else:
            s1 = list(parse_res['references']['direct inspiration'].keys())
            s2 = list(parse_res['references']['indirect inspiration'].keys())
            s3 = list(parse_res['references']['other inspiration'].keys())
            inspiration_types['s1'].extend(s1)
            inspiration_types['s2'].extend(s2)
            inspiration_types['s3'].extend(s3)
            
    except:
        s1 = parse_res['direct inspiration']
        s2 = parse_res['indirect inspiration']
        s3 = parse_res['other inspiration']
        inspiration_types['s1'].extend(s1)
        inspiration_types['s2'].extend(s2)
        inspiration_types['s3'].extend(s3)


    return inspiration_types
        



In [4]:
gpt4_result_train_level = {}
for k, v in total_prompt_train.items():
    res = []
    for ii in range(5):
        vv = v[ii]
        res.append(extract_references_2(vv))
    gpt4_result_train_level[k] = res


gpt4_result_valid_level = {}
for k, v in total_prompt_valid.items():
    res = []
    for ii in range(5):
        vv = v[ii]
        res.append(extract_references_2(vv))
    gpt4_result_valid_level[k] = res

gpt4_result_test_level = {}
for k, v in total_prompt_test.items():
    res = []
    for ii in range(5):
        vv = v[ii]
        res.append(extract_references_2(vv))
    gpt4_result_test_level[k] = res

In [5]:
import json
with open('gpt4_res_parse_train_level_parsed.json', 'w') as f:
    json.dump(gpt4_result_train_level, f)

import json
with open('gpt4_res_parse_valid_level_parsed.json', 'w') as f:
    json.dump(gpt4_result_valid_level, f)

import json
with open('gpt4_res_parse_test_level_parsed.json', 'w') as f:
    json.dump(gpt4_result_test_level, f)


In [151]:
### 第二部分数据

In [6]:
### 读数据 - label数据
import json
with open('gpt4_res_parse_train.json', 'rb') as f:
    total_prompt_train = json.load(f)

with open('gpt4_res_parse_valid.json', 'rb') as f:
    total_prompt_valid = json.load(f)

with open('gpt4_res_parse_test_short.json', 'rb') as f:
    total_prompt_test = json.load(f)


In [7]:
gpt4_res_parse_train_short = {}
for k, v in  total_prompt_train.items():
    res =  []
    for i in range(5):
        cleaned_string = v[i].strip('```json\n')
        res.append(json.loads(cleaned_string))
        gpt4_res_parse_train_short[k] = res

gpt4_res_parse_valid_short = {}
for k, v in  total_prompt_valid.items():
    res =  []
    for i in range(5):
        cleaned_string = v[i].strip('```json\n')
        res.append(json.loads(cleaned_string))
        gpt4_res_parse_valid_short[k] = res

gpt4_res_parse_test_short = {}
for k, v in  total_prompt_test.items():
    res =  []
    for i in range(5):
        cleaned_string = v[i].strip('```json\n')
        res.append(json.loads(cleaned_string))
        gpt4_res_parse_test_short[k] = res



In [8]:
import json
with open('gpt4_res_parse_train_short_parsed.json', 'w') as f:
    json.dump(gpt4_res_parse_train_short, f)

import json
with open('gpt4_res_parse_valid_short_parsed.json', 'w') as f:
    json.dump(gpt4_res_parse_valid_short, f)

import json
with open('gpt4_res_parse_test_short_parsed.json', 'w') as f:
    json.dump(gpt4_res_parse_test_short, f)


In [10]:
### OPUS数据
with open('opus_res_parse_test.json', 'rb') as f:
    total_prompt_test = json.load(f)

In [179]:
### 方式：让GPT4去提取结果
from openai import OpenAI
client = OpenAI(api_key= "YOUR_API_KEY", base_url="YOUR_BASE_URL")

def get_prompt_text(text):
    return f'''**** Your task is to extract the json answer from the text. The text is generated by a Large Language model to analyze an academic paper and return the import source papers. 
               **** Please return the result of the text in a json format, with the key is the reference number (Just keep the letter b and number like b1, b2...) and the value is the confidence score. The text is {text}'''

opus_res_parse_test = {}
for k, v in tqdm(total_prompt_test.items()):
    chat_completion = client.chat.completions.create(
        messages=[
          {  "role": "user",
            "content": get_prompt_text(v),
        }],
        n = 1,
        temperature=0.7,
        model="gpt-3.5-turbo")
    opus_res_parse_test[k] = chat_completion.choices[0].message.content

100%|██████████| 393/393 [10:56<00:00,  1.67s/it]


In [183]:
opus_res_parse_json_test = {}
for k, v in opus_res_parse_test.items():
    opus_res_parse_json_test[k] = eval(v)

import json
with open('opus_res_parse_json_test.json', 'w') as f:
    json.dump(opus_res_parse_json_test, f)

In [12]:
### GPT4-turbo数据
with open('gpt4_res_parse_test_only.json', 'rb') as f:
    total_prompt_test = json.load(f)

In [14]:
gpt4_turbo_parse_test = {}

for k, v in total_prompt_test.items():
    res = []
    for i in range(10):
        vv = v[i].strip("```json").strip("\n```")
        try:
            res.append(eval(vv))
        except:
            try:
                res.append(json.loads(vv))
            except:
                res.append({'-1':'-1'})
    gpt4_turbo_parse_test[k] = res


with open('gpt4_turbo_res_parse_json_test.json', 'w') as f:
    json.dump(gpt4_turbo_parse_test, f)

In [15]:
### GPT4-turbo数据 V2
with open('gpt4_res_parse_test_only_V2.json', 'rb') as f:
    total_prompt_test = json.load(f)

In [266]:
### 解析数据
turbo_res_v2 = {}

from openai import OpenAI
client = OpenAI(api_key= "YOUR_API_KEY", base_url="YOUR_BASE_URL")

def get_prompt_text(text):
    return f'''**** Your task is to extract the json answer from the text. The text is generated by a Large Language model to analyze an academic paper and return the import source papers. 
               **** Please return the result of the text in a json format, with the key is the reference number (Just keep the letter b and number like b1, b2...) and the value is the confidence score. The text is {text}'''

for k, v in tqdm(total_prompt_test.items()):
    res = []
    for ii in range(len(v)):
        try:
            vv = v[ii]
            chat_completion = client.chat.completions.create(
                messages=[
                {  "role": "user",
                    "content": get_prompt_text(vv),
                }],
                n = 1,
                temperature=0.7,
                model="gpt-3.5-turbo")
            res.append(chat_completion.choices[0].message.content)
        except:
            res.append({"-1":"-1"})
    turbo_res_v2[k] = res

  0%|          | 0/393 [00:00<?, ?it/s]

100%|██████████| 393/393 [2:27:46<00:00, 22.56s/it]  


In [323]:
turbo_res_v2_parse = {}
for k, v in turbo_res_v2.items():
    res = []
    for ii in range(10):
        try:
            vv = v[0].strip("```json").strip("\n```").split("\n```")[0]
            vvv = eval(vv)
            tmp = {}
            for aa, bb in vvv.items():
                aa = aa.strip('[').strip(']')
                tmp[aa] = bb
            res.append(tmp)

        except:
            res.append({"-1":"-1"})
    
    turbo_res_v2_parse[k] = res

In [324]:
with open('gpt4_turbo_res_parse_json_test_v2.json', 'w') as f:
    json.dump(turbo_res_v2_parse, f)

In [16]:
### GPT4-note数据

with open('GPT4_res_test_note.json', 'rb') as f:
    GPT4_res_test_note = json.load(f)

GPT4_res_test_note_parse = {}
for k, v in GPT4_res_test_note.items():
    res = []
    for ii in range(10):
        try:
            vv = v[ii].strip("```json").strip("\n```").split("\n```")[0]
            vvv = eval(vv)
            tmp = {}
            for aa, bb in vvv.items():
                aa = aa.strip('[').strip(']')
                tmp[aa] = bb
            res.append(tmp)

        except:
            res.append({"-1":"-1"})
    
    GPT4_res_test_note_parse[k] = res

In [17]:
with open('GPT4_res_test_note_parse.json', 'w') as f:
    json.dump(GPT4_res_test_note_parse, f)

In [18]:
### Parse gemini pro result 1
### gemini数据
with open('gemini_test_result_all.pkl', 'rb') as f:
    total_prompt_test = pickle.load(f)

with open('gemini_test_result_all_round2.pkl', 'rb') as f:
    total_prompt_test_round2 = pickle.load(f)

with open('gemini_test_result_all_round3.pkl', 'rb') as f:
    total_prompt_test_round3 = pickle.load(f)

with open('gemini_test_result_all_round4.pkl', 'rb') as f:
    total_prompt_test_round4 = pickle.load(f)

gemini_res_parse_json_test = {}
gemini_res_parse_json_test_round2 = {}
gemini_res_parse_json_test_round3 = {}
gemini_res_parse_json_test_round4 = {}

for k, v in total_prompt_test.items():
    try:
        vv = v[0].strip("```json").strip("\n```").split("\n```")[0]
        vvv = eval(vv)
        tmp = {}
        for aa, bb in vvv.items():
            aa = aa.strip('[').strip(']')
            tmp[aa] = bb
        gemini_res_parse_json_test[k] = tmp

    except:
        gemini_res_parse_json_test[k] = {"-1":"-1"}
    
    try:
        v = total_prompt_test_round2[k]
        vv = v[0].strip("```json").strip("\n```").split("\n```")[0]
        vvv = eval(vv)
        tmp = {}
        for aa, bb in vvv.items():
            aa = aa.strip('[').strip(']')
            tmp[aa] = bb
        gemini_res_parse_json_test_round2[k] = tmp

    except:
        gemini_res_parse_json_test_round2[k] = {"-1":"-1"}

    try:
        v = total_prompt_test_round3[k]
        vv = v[0].strip("```json").strip("\n```").split("\n```")[0]
        vvv = eval(vv)
        tmp = {}
        for aa, bb in vvv.items():
            aa = aa.strip('[').strip(']')
            tmp[aa] = bb
        gemini_res_parse_json_test_round3[k] = tmp

    except:
        gemini_res_parse_json_test_round3[k] = {"-1":"-1"}

    try:
        v = total_prompt_test_round4[k]
        vv = v[0].strip("```json").strip("\n```").split("\n```")[0]
        vvv = eval(vv)
        tmp = {}
        for aa, bb in vvv.items():
            aa = aa.strip('[').strip(']')
            tmp[aa] = float(bb)
        gemini_res_parse_json_test_round4[k] = tmp

    except:
        gemini_res_parse_json_test_round4[k] = {"-1":"-1"}

with open('gemini_res_parse_json_test.json', 'w') as f:
    json.dump(gemini_res_parse_json_test, f)

with open('gemini_res_parse_json_test_round2.json', 'w') as f:
    json.dump(gemini_res_parse_json_test_round2, f)

with open('gemini_res_parse_json_test_round3.json', 'w') as f:
    json.dump(gemini_res_parse_json_test_round3, f)

with open('gemini_res_parse_json_test_round4.json', 'w') as f:
    json.dump(gemini_res_parse_json_test_round4, f)


In [332]:
### Parse data
with open('GPT4_res_test_V3_save.json', 'rb') as f:
    GPT4_res_test_V3_save = json.load(f)

GPT4_res_test_V3_save_parse ={}

In [333]:
client = OpenAI(api_key= "YOUR_API_KEY", base_url="YOUR_BASE_URL")

def get_prompt_text(text):
    return f'''**** Your task is to extract the json answer from the text. The text is generated by a Large Language model to analyze an academic paper and return the import source papers. 
               **** Please return the result of the text in a json format, with the key is the reference number (Just keep the letter b and number like b1, b2...) and the value is the confidence score. The text is {text}'''

for k, v in tqdm(GPT4_res_test_V3_save.items()):
    res = []
    for ii in range(len(v)):
        vv = v[ii]
        chat_completion = client.chat.completions.create(
                messages=[
                {  "role": "user",
                    "content": get_prompt_text(vv),
                }],
                n = 1,
                temperature=0.7,
                model="gpt-3.5-turbo")
        res.append(chat_completion.choices[0].message.content)

    GPT4_res_test_V3_save_parse[k] = res

  0%|          | 0/393 [00:00<?, ?it/s]

100%|██████████| 393/393 [1:14:56<00:00, 11.44s/it]


In [19]:
GPT4_res_test_V3_parse_json = {}
for k, v in GPT4_res_test_V3_save_parse.items():
    this_res = []
    for ii in range(len(v)):
        try:
            vv = v[ii].strip("```json").strip("\n```").split("\n```")[0]
            vvv = eval(vv)
            this_res.append(vvv)
        except:
            this_res.append({"-1":"-1"})
    
    GPT4_res_test_V3_parse_json[k] = this_res

NameError: name 'GPT4_res_test_V3_save_parse' is not defined

In [343]:
## save to json
## save to json
with open('GPT4_res_test_V3_parse.json', 'w') as f:
    json.dump(GPT4_res_test_V3_parse_json, f)